In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from ipywidgets import FloatProgress

In [6]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
# /opt/ml/code/mmdetection_trash/configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_2x_coco.py
# mask
# /opt/ml/code/mmdetection_trash/configs/mask_rcnn/mask_rcnn_r50_fpn_2x_coco.py
cfg = Config.fromfile('/opt/ml/code/mmdetection_trash/configs/trash/mask_rcnn/mask_rcnn_r50_fpn_1x_trash.py')

#/opt/ml/input/data/train.json
PREFIX = '/opt/ml/input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

# CUSTOM
cfg.checkpoint_config = dict(interval=1, max_keep_ckpts=3)
#cfg.optimizer = dict(type='Adam', lr=0.001, weight_decay=0.0001)
'''
cfg.lr_config = dict(
    policy='CosineRestart',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=1.0 / 10,
    periods=[12],
    restart_weights = [1],
    min_lr_ratio=1e-5)
'''
cfg.model.roi_head.bbox_head.num_classes = 11
cfg.model.roi_head.mask_head.num_classes = 11

cfg.seed=24
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/mask_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [7]:
model = build_detector(cfg.model)

2021-05-14 13:49:14,083 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-14 13:49:14,084 - mmdet - INFO - Use load_from_torchvision loader
2021-05-14 13:49:14,335 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [8]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.63s)
creating index...
index created!


In [9]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...
2021-05-14 13:49:19,051 - mmdet - INFO - Start running, host: root@f8a15f3ed9af, work_dir: /opt/ml/code/mmdetection_trash/work_dirs/mask_rcnn_r50_fpn_1x_trash
Done (t=0.82s)
creating index...
index created!
2021-05-14 13:49:19,052 - mmdet - INFO - workflow: [('train', 1)], max: 36 epochs
2021-05-14 13:49:41,202 - mmdet - INFO - Epoch [1][50/655]	lr: 1.978e-03, eta: 2:53:41, time: 0.443, data_time: 0.066, memory: 2891, loss_rpn_cls: 0.6075, loss_rpn_bbox: 0.0896, loss_cls: 0.7146, acc: 85.5381, loss_bbox: 0.1767, loss_mask: 0.8619, loss: 2.4503, grad_norm: 8.2252
2021-05-14 13:50:02,060 - mmdet - INFO - Epoch [1][100/655]	lr: 3.976e-03, eta: 2:48:17, time: 0.417, data_time: 0.020, memory: 3242, loss_rpn_cls: 0.2337, loss_rpn_bbox: 0.0808, loss_cls: 0.4430, acc: 90.6172, loss_bbox: 0.3476, loss_mask: 0.6740, loss: 1.7792, grad_norm: 3.3702
2021-05-14 13:50:23,044 - mmdet - INFO - Epoch [1][150/655]	lr: 5.974e-03, eta: 2:46:34, time: 0.420, data_time: 0.

IndexError: list index out of range